<a href="https://colab.research.google.com/github/anujtech313/Machine-Learning/blob/master/xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this Project I will use Regression xgboost model and getting best parameters using optuna library.

In [6]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import optuna

In [8]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import optuna
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [11]:
df = pd.read_csv('/content/Admission_Prediction.csv')

In [12]:
df

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337.0,118.0,4.0,4.5,4.5,9.65,1,0.92
1,2,324.0,107.0,4.0,4.0,4.5,8.87,1,0.76
2,3,NaN,104.0,3.0,3.0,3.5,8.00,1,0.72
3,4,322.0,110.0,3.0,3.5,2.5,8.67,1,0.80
4,5,314.0,103.0,2.0,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...,...
495,496,332.0,108.0,5.0,4.5,4.0,9.02,1,0.87
496,497,337.0,117.0,5.0,5.0,5.0,9.87,1,0.96
497,498,330.0,120.0,5.0,4.5,5.0,9.56,1,0.93
498,499,312.0,103.0,4.0,4.0,5.0,8.43,0,0.73


In [13]:
x = df.drop(columns=['Serial No.', 'Chance of Admit'])
x

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research
0,337.0,118.0,4.0,4.5,4.5,9.65,1
1,324.0,107.0,4.0,4.0,4.5,8.87,1
2,NaN,104.0,3.0,3.0,3.5,8.00,1
3,322.0,110.0,3.0,3.5,2.5,8.67,1
4,314.0,103.0,2.0,2.0,3.0,8.21,0
...,...,...,...,...,...,...,...
495,332.0,108.0,5.0,4.5,4.0,9.02,1
496,337.0,117.0,5.0,5.0,5.0,9.87,1
497,330.0,120.0,5.0,4.5,5.0,9.56,1
498,312.0,103.0,4.0,4.0,5.0,8.43,0


In [14]:
y = df['Chance of Admit']

In [15]:
df['GRE Score'] = df['GRE Score'].fillna(df['GRE Score'].mean())
df['TOEFL Score'] = df['TOEFL Score'].fillna(df['TOEFL Score'].mean())
df['University Rating'] = df['University Rating'].fillna(df['University Rating'].mean())

In [16]:
from sklearn.preprocessing import StandardScaler
std_sca = StandardScaler()
x = std_sca.fit_transform(x)

In [17]:
x

array([[ 1.81488957,  1.77056647,  0.7671331 , ...,  1.09894429,
         1.77680627,  0.88640526],
       [ 0.66067545, -0.03074596,  0.7671331 , ...,  1.09894429,
         0.48585943,  0.88640526],
       [        nan, -0.52201299, -0.10624613, ...,  0.01730621,
        -0.95404281,  0.88640526],
       ...,
       [ 1.19338966,  2.09807783,  1.64051233, ...,  1.63976333,
         1.62785086,  0.88640526],
       [-0.40475296, -0.68576867,  0.7671331 , ...,  1.63976333,
        -0.24236699, -1.12815215],
       [ 0.92703256,  0.95178809,  0.7671331 , ...,  1.09894429,
         0.76721964, -1.12815215]])

In [18]:
from sklearn.metrics import mean_squared_error
def objective(trail , data = x , target = y ):
    train_x , test_x ,train_y , test_y = train_test_split(data , target , test_size = .10,random_state = 30  )
    param = {
        'tree_method':'auto',
        'lambda':trail.suggest_loguniform('lambda' , 1e-4,10.0),
        'alpha' :trail.suggest_loguniform('alpha' , 1e-4 , 10.0),
        'colsample_bytree' :trail.suggest_categorical('colsample_bytree' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'subsample' :trail.suggest_categorical('subsample' , [.1,.2,.3,.4,.5,.6,.7,.8,.9,1]),
        'learning_rate' : trail.suggest_categorical('learning_rate' , [.00001,.0003,.008,.02,.01,1,8]),
        'n_estimator' :30000,
        'max_depth' :trail.suggest_categorical('max_depth', [3,4,5,6,7,8,9,10,11,12]),
        'random_state' :trail.suggest_categorical('random_state' , [10,20,30 ,2000 , 3454,243123]),
        'min_child_weight' :trail.suggest_int('min_child_weight' ,1,200)
    }
    xgb_reg_model = xgb.XGBRegressor(**param)
    xgb_reg_model.fit(train_x,train_y,eval_set = [(test_x,test_y)] , verbose = True )
    pred_xgb = xgb_reg_model.predict(test_x)
    rmse = mean_squared_error(test_y,pred_xgb)
    return rmse

In [19]:
xgb.XGBRFRegressor()

XGBRFRegressor()

In [20]:
find_param = optuna.create_study(direction='minimize')
find_param.optimize(objective,n_trials = 10)
find_param.best_trial.params

[I 2022-10-11 13:54:21,919] A new study created in memory with name: no-name-6b3fde4c-93a9-4fb5-8fd6-08e3081ca100
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  import sys


[13:54:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.230834
[1]	validation_0-rmse:0.226374
[2]	validation_0-rmse:0.22223
[3]	validation_0-rmse:0.218118
[4]	validation_0-rmse:0.214395
[5]	validation_0-rmse:0.21048
[6]	validation_0-rmse:0.206805
[7]	validation_0-rmse:0.203233
[8]	validation_0-rmse:0.199711
[9]	validation_0-rmse:0.196258
[10]	validation_0-rmse:0.192934
[11]	validation_0-rmse:0.189392
[12]	validation_0-rmse:0.18602
[13]	validation_0-rmse:0.18246
[14]	validation_0-rmse:0.179464
[15]	validation_0-rmse:0.176302
[16]	validation_0-rmse:0.17316
[17]	validation_0-rmse:0.169954
[18]	validation_0-rmse:0.167076
[19]	validation_0-rmse:0.164226
[20]	validation_0-rmse:0.161378
[21]	validation_0-rmse:0.158538
[22]	validation_0-rmse:0.155837
[23]	validation_0-rmse:0.153365
[24]	validation_0-rmse:0.150826
[25]	validation_0-rmse:0.148293
[26]	validation_0-rmse:0.145908
[27]	validation_0-rmse:0

[I 2022-10-11 13:54:22,813] Trial 0 finished with value: 0.00466765953375959 and parameters: {'lambda': 0.04405888825682249, 'alpha': 0.008606081916463617, 'colsample_bytree': 0.9, 'subsample': 0.2, 'learning_rate': 0.02, 'max_depth': 10, 'random_state': 20, 'min_child_weight': 7}. Best is trial 0 with value: 0.00466765953375959.


[13:54:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.072443
[1]	validation_0-rmse:0.07652
[2]	validation_0-rmse:0.07541
[3]	validation_0-rmse:0.076087
[4]	validation_0-rmse:0.075157
[5]	validation_0-rmse:0.073158
[6]	validation_0-rmse:0.07958
[7]	validation_0-rmse:0.076103
[8]	validation_0-rmse:0.078756
[9]	validation_0-rmse:0.078923
[10]	validation_0-rmse:0.078075
[11]	validation_0-rmse:0.078315
[12]	validation_0-rmse:0.078361
[13]	validation_0-rmse:0.078472
[14]	validation_0-rmse:0.076207
[15]	validation_0-rmse:0.075458
[16]	validation_0-rmse:0.077348
[17]	validation_0-rmse:0.080382
[18]	validation_0-rmse:0.080446
[19]	validation_0-rmse:0.078959
[20]	validation_0-rmse:0.078299
[21]	validation_0-rmse:0.077367
[22]	validation_0-rmse:0.077371
[23]	validation_0-rmse:0.079358
[24]	validation_0-rmse:0.078207
[25]	validation_0-rmse:0.077983
[26]	validation_0-rmse:0.078009
[27]	validation_0-rmse

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  import sys


...

[13:54:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.23343
[1]	validation_0-rmse:0.231703
[2]	validation_0-rmse:0.230002
[3]	validation_0-rmse:0.228515
[4]	validation_0-rmse:0.226787
[5]	validation_0-rmse:0.225046
[6]	validation_0-rmse:0.223279
[7]	validation_0-rmse:0.221519
[8]	validation_0-rmse:0.219937
[9]	validation_0-rmse:0.218294
[10]	validation_0-rmse:0.216925
[11]	validation_0-rmse:0.215509
[12]	validation_0-rmse:0.214156


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  import sys


[13]	validation_0-rmse:0.212692
[14]	validation_0-rmse:0.211329
[15]	validation_0-rmse:0.210034
[16]	validation_0-rmse:0.208606
[17]	validation_0-rmse:0.207294
[18]	validation_0-rmse:0.205776
[19]	validation_0-rmse:0.204108
[20]	validation_0-rmse:0.202608
[21]	validation_0-rmse:0.201251
[22]	validation_0-rmse:0.199784
[23]	validation_0-rmse:0.198428
[24]	validation_0-rmse:0.197183
[25]	validation_0-rmse:0.19575
[26]	validation_0-rmse:0.194582
[27]	validation_0-rmse:0.193363
[28]	validation_0-rmse:0.192297
[29]	validation_0-rmse:0.191005
[30]	validation_0-rmse:0.189878
[31]	validation_0-rmse:0.188777
[32]	validation_0-rmse:0.18766
[33]	validation_0-rmse:0.186461
[34]	validation_0-rmse:0.185393
[35]	validation_0-rmse:0.184014
[36]	validation_0-rmse:0.182975
[37]	validation_0-rmse:0.181781
[38]	validation_0-rmse:0.180824
[39]	validation_0-rmse:0.179741
[40]	validation_0-rmse:0.178513
[41]	validation_0-rmse:0.177528
[42]	validation_0-rmse:0.176547
[43]	validation_0-rmse:0.175551
[44]	valid

[I 2022-10-11 13:54:25,359] Trial 9 finished with value: 0.02065408760495614 and parameters: {'lambda': 0.00010042139528304351, 'alpha': 0.006490646621944349, 'colsample_bytree': 0.4, 'subsample': 0.1, 'learning_rate': 0.01, 'max_depth': 7, 'random_state': 30, 'min_child_weight': 75}. Best is trial 0 with value: 0.00466765953375959.


{'lambda': 0.04405888825682249,
 'alpha': 0.008606081916463617,
 'colsample_bytree': 0.9,
 'subsample': 0.2,
 'learning_rate': 0.02,
 'max_depth': 10,
 'random_state': 20,
 'min_child_weight': 7}

In [21]:
find_param.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_alpha,params_colsample_bytree,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_random_state,params_subsample,state
0,0,0.004668,2022-10-11 13:54:21.925878,2022-10-11 13:54:22.812369,0 days 00:00:00.886491,0.008606,0.9,0.044059,0.02000,10,7,20,0.2,COMPLETE
1,1,0.007823,2022-10-11 13:54:22.815441,2022-10-11 13:54:23.048965,0 days 00:00:00.233524,0.023496,0.9,0.186373,1.00000,8,34,20,0.8,COMPLETE
2,2,0.052706,2022-10-11 13:54:23.051196,2022-10-11 13:54:23.242640,0 days 00:00:00.191444,0.343169,0.7,0.000154,0.00030,5,78,10,0.3,COMPLETE
3,3,0.013819,2022-10-11 13:54:23.245801,2022-10-11 13:54:23.475146,0 days 00:00:00.229345,2.349774,0.9,0.012795,0.00800,5,12,3454,0.3,COMPLETE
4,4,0.010651,2022-10-11 13:54:23.477549,2022-10-11 13:54:23.915771,0 days 00:00:00.438222,0.002373,0.6,0.066343,0.01000,12,94,10,1.0,COMPLETE
5,5,0.055133,2022-10-11 13:54:23.917824,2022-10-11 13:54:24.100842,0 days 00:00:00.183018,0.992232,1.0,0.014267,0.00001,11,134,2000,0.3,COMPLETE
6,6,0.020646,2022-10-11 13:54:24.102846,2022-10-11 13:54:24.270601,0 days 00:00:00.167755,0.006178,0.1,0.002218,0.01000,7,147,20,0.2,COMPLETE
7,7,0.004879,2022-10-11 13:54:24.272931,2022-10-11 13:54:24.745528,0 days 00:00:00.472597,0.036186,0.9,0.069962,0.02000,12,66,30,1.0,COMPLETE
8,8,0.020479,2022-10-11 13:54:24.747485,2022-10-11 13:54:24.913838,0 days 00:00:00.166353,0.001860,0.6,1.285403,0.01000,8,196,2000,0.8,COMPLETE
9,9,0.020654,2022-10-11 13:54:24.915955,2022-10-11 13:54:25.358722,0 days 00:00:00.442767,0.006491,0.4,0.000100,0.01000,7,75,30,0.1,COMPLETE


In [22]:
optuna.visualization.plot_optimization_history(find_param)

In [23]:
optuna.visualization.plot_slice(find_param)

In [24]:
optuna.visualization.plot_contour(find_param,params=['alpha','lambda'])

In [25]:
best_param = {'lambda': 0.0011060303717433694,
 'alpha': 0.30894702055614454,
 'colsample_bytree': 0.6,
 'subsample': 0.5,
 'learning_rate': 0.02,
 'max_depth': 5,
 'random_state': 20,
 'min_child_weight': 22}

In [26]:
xgb_final_model =  xgb.XGBRegressor(**best_param)

In [27]:
xgb_final_model.fit(x,y)

[13:54:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=0.30894702055614454, colsample_bytree=0.6,
             lambda=0.0011060303717433694, learning_rate=0.02, max_depth=5,
             min_child_weight=22, random_state=20, subsample=0.5)

In [28]:
xgb_final_model.score(x,y)

0.7638146364461023

# Classification 

In [30]:
df = pd.read_csv('winequality-red.csv')

In [31]:
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [32]:
x = df.drop(columns='quality')

In [33]:
y = df.quality

In [44]:
train_x , test_x , train_y , test_y= train_test_split(x , y , test_size = .20 )


In [45]:
np.asarray(train_x)

array([[ 5.6  ,  0.5  ,  0.09 , ...,  3.63 ,  0.63 , 13.   ],
       [ 7.2  ,  0.695,  0.13 , ...,  3.29 ,  0.54 , 10.1  ],
       [ 8.9  ,  0.635,  0.37 , ...,  3.   ,  1.09 ,  9.3  ],
       ...,
       [ 4.9  ,  0.42 ,  0.   , ...,  3.71 ,  0.74 , 14.   ],
       [10.4  ,  0.41 ,  0.55 , ...,  3.15 ,  0.89 ,  9.9  ],
       [ 6.9  ,  0.48 ,  0.2  , ...,  3.39 ,  0.43 ,  9.05 ]])

In [46]:
xgb_classification = xgb.XGBClassifier()
xgb_classification.fit(train_x , train_y,eval_set  = [(test_x, test_y)])

[0]	validation_0-merror:0.425
[1]	validation_0-merror:0.428125
[2]	validation_0-merror:0.41875
[3]	validation_0-merror:0.415625
[4]	validation_0-merror:0.415625
[5]	validation_0-merror:0.415625
[6]	validation_0-merror:0.41875
[7]	validation_0-merror:0.409375
[8]	validation_0-merror:0.40625
[9]	validation_0-merror:0.396875
[10]	validation_0-merror:0.390625
[11]	validation_0-merror:0.384375
[12]	validation_0-merror:0.3875
[13]	validation_0-merror:0.3875
[14]	validation_0-merror:0.3875
[15]	validation_0-merror:0.384375
[16]	validation_0-merror:0.38125
[17]	validation_0-merror:0.378125
[18]	validation_0-merror:0.38125
[19]	validation_0-merror:0.378125
[20]	validation_0-merror:0.38125
[21]	validation_0-merror:0.378125
[22]	validation_0-merror:0.378125
[23]	validation_0-merror:0.378125
[24]	validation_0-merror:0.378125
[25]	validation_0-merror:0.375
[26]	validation_0-merror:0.371875
[27]	validation_0-merror:0.375
[28]	validation_0-merror:0.384375
[29]	validation_0-merror:0.375
[30]	validatio

XGBClassifier(objective='multi:softprob')

In [53]:
def objective_classification(trial) :
    train_x , test_x , train_y , test_y= train_test_split(x , y , test_size = .20 )
    
    param  = {
        'tree_method' :'gpu_hist',
        'verbosity' : 3,
        'objective' : "binary:logistics" , 
        'booster' : trial.suggest_categorical('booster' , ['dart' , 'gbtree','gblinear']),
        'lambda' : trial.suggest_float('lambda' , 1e-4 , 1),
        'alpha' :trial.suggest_float('alpha' , 1e-4 , 1),
        'subsample' : trial.suggest_float('subsample' , .1,.5),
        'colsample_bytree' : trial.suggest_float('colsample_bytree' , .1 ,.5)
        
    }
   
    if param['booster'] in ['gbtree' , 'dart']:
        param['gamma'] :trial.suggest_float('gamma' , 1e-3 , 4 )
        param['eta'] : trial.suggest_float('eta' , .001 ,5 )
        
    xgb_classification = xgb.XGBClassifier(**param)
    xgb_classification.fit(train_x , train_y,eval_set  = [(test_x, test_y)])
    pred = xgb_classification.predict(test_x)
    accuracy  = xgb_classification.score(test_x , test_y)

    
    
    return accuracy
                         

In [54]:
xgb_classification_oputuna = optuna.create_study(direction  = 'minimize')
xgb_classification_oputuna.optimize(objective_classification , n_trials = 10 )

[I 2022-10-11 15:29:13,936] A new study created in memory with name: no-name-b41a5e1e-c0c6-4723-b482-a6501ad02131


[0]	validation_0-merror:0.546875
[1]	validation_0-merror:0.534375
[2]	validation_0-merror:0.515625
[3]	validation_0-merror:0.496875
[4]	validation_0-merror:0.4875
[5]	validation_0-merror:0.48125
[6]	validation_0-merror:0.4625
[7]	validation_0-merror:0.45
[8]	validation_0-merror:0.45
[9]	validation_0-merror:0.465625
[10]	validation_0-merror:0.478125
[11]	validation_0-merror:0.475
[12]	validation_0-merror:0.471875
[13]	validation_0-merror:0.475
[14]	validation_0-merror:0.475
[15]	validation_0-merror:0.475
[16]	validation_0-merror:0.475
[17]	validation_0-merror:0.475
[18]	validation_0-merror:0.475
[19]	validation_0-merror:0.471875
[20]	validation_0-merror:0.46875
[21]	validation_0-merror:0.46875
[22]	validation_0-merror:0.475
[23]	validation_0-merror:0.475
[24]	validation_0-merror:0.475
[25]	validation_0-merror:0.475
[26]	validation_0-merror:0.475
[27]	validation_0-merror:0.475
[28]	validation_0-merror:0.475
[29]	validation_0-merror:0.475
[30]	validation_0-merror:0.471875
[31]	validation_

[I 2022-10-11 15:29:14,425] Trial 0 finished with value: 0.53125 and parameters: {'booster': 'gblinear', 'lambda': 0.18593471936969888, 'alpha': 0.307652776904251, 'subsample': 0.12951633919182917, 'colsample_bytree': 0.11390762486229473}. Best is trial 0 with value: 0.53125.


[15:29:14] PredictBatch: 0.00624492s, 202 calls @ 30us
[15:29:14] PredictBatchInternal: 0.183727s, 203 calls @ 905us
[0]	validation_0-merror:0.56875
[1]	validation_0-merror:0.56875
[2]	validation_0-merror:0.5625
[3]	validation_0-merror:0.5625
[4]	validation_0-merror:0.56875
[5]	validation_0-merror:0.565625
[6]	validation_0-merror:0.559375
[7]	validation_0-merror:0.54375
[8]	validation_0-merror:0.540625
[9]	validation_0-merror:0.534375
[10]	validation_0-merror:0.540625
[11]	validation_0-merror:0.55
[12]	validation_0-merror:0.55
[13]	validation_0-merror:0.540625
[14]	validation_0-merror:0.540625
[15]	validation_0-merror:0.540625
[16]	validation_0-merror:0.5375
[17]	validation_0-merror:0.540625
[18]	validation_0-merror:0.534375
[19]	validation_0-merror:0.5375
[20]	validation_0-merror:0.5375
[21]	validation_0-merror:0.5375
[22]	validation_0-merror:0.5375
[23]	validation_0-merror:0.5375
[24]	validation_0-merror:0.5375
[25]	validation_0-merror:0.5375
[26]	validation_0-merror:0.5375
[27]	vali

[I 2022-10-11 15:29:14,933] Trial 1 finished with value: 0.484375 and parameters: {'booster': 'gblinear', 'lambda': 0.2215938439957984, 'alpha': 0.3436736227693487, 'subsample': 0.3382863499705036, 'colsample_bytree': 0.2087195327916184}. Best is trial 1 with value: 0.484375.


...

[15:29:27] InitDataOnce: 0.0158498s, 1 calls @ 15849us
[15:29:27] Quantiles: 0.00163942s, 1 calls @ 1639us
[15:29:27] Update: 1.91152s, 600 calls @ 3185us
[15:29:27] ======== NCCL Statistics========
[15:29:27] AllReduce calls: 2867
[15:29:27] AllReduce total MB communicated: 42
[15:29:27] ======== Monitor: HistCutMatrix ========
[15:29:27] ======== Monitor: DeviceShard0 ========
[15:29:27] AllReduce: 0.00890729s, 2267 calls @ 3us
[15:29:27] BuildHist: 0.0415381s, 1667 calls @ 24us
[15:29:27] EvaluateSplits: 0.0756241s, 1667 calls @ 45us
[15:29:27] FinalisePosition: 0.00568913s, 600 calls @ 9us
[15:29:27] InitRoot: 1.66218s, 600 calls @ 2770us
[15:29:27] Reset: 0.0658837s, 600 calls @ 109us
[15:29:27] UpdatePosition: 0.0376224s, 1667 calls @ 22us
[15:29:27] ======== Monitor: HistCutMatrix ========
[15:29:27] INFO: /workspace/src/gbm/gbtree.cc:479: drop 0 trees, weight = 1
[0]	validation_0-merror:0.553125
[15:29:27] INFO: /workspace/src/gbm/gbtree.cc:479: drop 0 trees, weight = 1
[1]	val

[I 2022-10-11 15:29:31,814] Trial 8 finished with value: 0.621875 and parameters: {'booster': 'dart', 'lambda': 0.2353342352823329, 'alpha': 0.6938220770074054, 'subsample': 0.40666460038873364, 'colsample_bytree': 0.3243433338050563}. Best is trial 1 with value: 0.484375.


[15:29:31] ======== Monitor: DeviceShard0 ========
[15:29:31] AllReduce: 0.00762747s, 2232 calls @ 3us
[15:29:31] BuildHist: 0.0437358s, 1632 calls @ 26us
[15:29:31] EvaluateSplits: 0.0713633s, 1632 calls @ 43us
[15:29:31] FinalisePosition: 0.00531433s, 600 calls @ 8us
[15:29:31] InitRoot: 1.6595s, 600 calls @ 2765us
[15:29:31] Reset: 0.0575641s, 600 calls @ 95us
[15:29:31] UpdatePosition: 0.0346872s, 1632 calls @ 21us
[15:29:31] ======== Monitor: HistCutMatrix ========
[0]	validation_0-merror:0.565625
[1]	validation_0-merror:0.553125
[2]	validation_0-merror:0.534375
[3]	validation_0-merror:0.54375
[4]	validation_0-merror:0.525
[5]	validation_0-merror:0.503125
[6]	validation_0-merror:0.5
[7]	validation_0-merror:0.48125
[8]	validation_0-merror:0.4875
[9]	validation_0-merror:0.49375
[10]	validation_0-merror:0.4875
[11]	validation_0-merror:0.48125
[12]	validation_0-merror:0.490625
[13]	validation_0-merror:0.490625
[14]	validation_0-merror:0.4875
[15]	validation_0-merror:0.471875
[16]	vali

[I 2022-10-11 15:29:33,927] Trial 9 finished with value: 0.590625 and parameters: {'booster': 'gbtree', 'lambda': 0.1607646392220734, 'alpha': 0.028086598469874186, 'subsample': 0.29375867880500084, 'colsample_bytree': 0.14476482672610202}. Best is trial 1 with value: 0.484375.


[15:29:33] EvalOneIter: 0.00616551s, 100 calls @ 61us
[15:29:33] GetGradient: 0.00475127s, 100 calls @ 47us
[15:29:33] PredictRaw: 0.00289191s, 100 calls @ 28us
[15:29:33] UpdateOneIter: 1.93623s, 100 calls @ 19362us
[15:29:33] ======== Monitor: GBTree ========
[15:29:33] BoostNewTrees: 1.92009s, 100 calls @ 19200us
[15:29:33] CommitModel: 0.00792311s, 100 calls @ 79us
[15:29:33] ======== Monitor:  ========
[15:29:33] DevicePredictInternal: 0.00807516s, 202 calls @ 39us
[15:29:33] PredictFromCache: 0.00357886s, 199 calls @ 17us
[15:29:33] ======== Monitor: updater_gpu_hist ========
[15:29:33] BinningCompression: 0.000360185s, 1 calls @ 360us
[15:29:33] InitData: 0.00957954s, 600 calls @ 15us
[15:29:33] InitDataOnce: 0.00944958s, 1 calls @ 9449us
[15:29:33] Quantiles: 0.00141406s, 1 calls @ 1414us
[15:29:33] Update: 1.89129s, 600 calls @ 3152us
[15:29:33] ======== NCCL Statistics========
[15:29:33] AllReduce calls: 2608
[15:29:33] AllReduce total MB communicated: 37
[15:29:33] ======== 

In [55]:
xgb_classification_oputuna.best_trial

FrozenTrial(number=1, values=[0.484375], datetime_start=datetime.datetime(2022, 10, 11, 15, 29, 14, 427419), datetime_complete=datetime.datetime(2022, 10, 11, 15, 29, 14, 932852), params={'booster': 'gblinear', 'lambda': 0.2215938439957984, 'alpha': 0.3436736227693487, 'subsample': 0.3382863499705036, 'colsample_bytree': 0.2087195327916184}, distributions={'booster': CategoricalDistribution(choices=('dart', 'gbtree', 'gblinear')), 'lambda': FloatDistribution(high=1.0, log=False, low=0.0001, step=None), 'alpha': FloatDistribution(high=1.0, log=False, low=0.0001, step=None), 'subsample': FloatDistribution(high=0.5, log=False, low=0.1, step=None), 'colsample_bytree': FloatDistribution(high=0.5, log=False, low=0.1, step=None)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=1, state=TrialState.COMPLETE, value=None)

In [56]:
param = xgb_classification_oputuna.best_trial.params

In [57]:
xgb_final_clas = xgb.XGBClassifier(**param)
xgb_final_clas.fit(train_x , train_y)

XGBClassifier(alpha=0.3436736227693487, booster='gblinear',
              colsample_bytree=0.2087195327916184, lambda=0.2215938439957984,
              objective='multi:softprob', subsample=0.3382863499705036)

In [58]:
xgb_final_clas.score(test_x , test_y)

0.54375